# Jigsaw Agile Community Rules Classification

This notebook implements multiple machine learning models to achieve the highest AUC score (target ≥0.95) for the Jigsaw Agile Community Rules Classification hackathon.

## Dataset Information
- **Train data**: Contains rule violations with positive/negative examples
- **Test data**: Similar structure but without rule_violation labels
- **Evaluation metric**: Column-averaged AUC

## Models to Test:
1. Logistic Regression on TF-IDF
2. Random Forest Classifier on TF-IDF
3. XGBoost Classifier on TF-IDF
4. SVM on TF-IDF
5. MLP (PyTorch) on TF-IDF embeddings


In [3]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("Libraries imported successfully!")


Libraries imported successfully!


In [4]:
# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print(f"Train dataset shape: {train_df.shape}")
print(f"Test dataset shape: {test_df.shape}")
print(f"\nTrain dataset columns: {train_df.columns.tolist()}")
print(f"\nTrain dataset info:")
print(train_df.info())
print(f"\nFirst few rows of train data:")
print(train_df.head())


Train dataset shape: (2029, 9)
Test dataset shape: (10, 8)

Train dataset columns: ['row_id', 'body', 'rule', 'subreddit', 'positive_example_1', 'positive_example_2', 'negative_example_1', 'negative_example_2', 'rule_violation']

Train dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB
None

First few rows of train data:
   row_id                   

In [5]:
# Data augmentation: Add positive and negative examples
def augment_training_data(df):
    """
    Augment training data by adding positive_example_1/2 as positive samples (rule_violation=1)
    and negative_example_1/2 as negative samples (rule_violation=0)
    """
    augmented_data = []
    
    # Add original data
    for _, row in df.iterrows():
        augmented_data.append({
            'body': row['body'],
            'rule': row['rule'],
            'subreddit': row['subreddit'],
            'rule_violation': row['rule_violation']
        })
    
    # Add positive examples (rule_violation=1)
    for _, row in df.iterrows():
        if pd.notna(row['positive_example_1']):
            augmented_data.append({
                'body': row['positive_example_1'],
                'rule': row['rule'],
                'subreddit': row['subreddit'],
                'rule_violation': 1
            })
        
        if pd.notna(row['positive_example_2']):
            augmented_data.append({
                'body': row['positive_example_2'],
                'rule': row['rule'],
                'subreddit': row['subreddit'],
                'rule_violation': 1
            })
    
    # Add negative examples (rule_violation=0)
    for _, row in df.iterrows():
        if pd.notna(row['negative_example_1']):
            augmented_data.append({
                'body': row['negative_example_1'],
                'rule': row['rule'],
                'subreddit': row['subreddit'],
                'rule_violation': 0
            })
        
        if pd.notna(row['negative_example_2']):
            augmented_data.append({
                'body': row['negative_example_2'],
                'rule': row['rule'],
                'subreddit': row['subreddit'],
                'rule_violation': 0
            })
    
    return pd.DataFrame(augmented_data)

# Augment the training data
augmented_train = augment_training_data(train_df)
print(f"Original train data size: {len(train_df)}")
print(f"Augmented train data size: {len(augmented_train)}")
print(f"\nClass distribution in augmented data:")
print(augmented_train['rule_violation'].value_counts())


Original train data size: 2029
Augmented train data size: 10145

Class distribution in augmented data:
rule_violation
1    5089
0    5056
Name: count, dtype: int64


In [6]:
# Feature engineering: Add has_url and body_length features
def add_features(df):
    """
    Add 'has_url' and 'body_length' features to the dataframe
    """
    df = df.copy()
    
    # has_url: 1 if 'http' or 'www' in body, else 0
    df['has_url'] = df['body'].apply(lambda x: 1 if ('http' in str(x).lower() or 'www' in str(x).lower()) else 0)
    
    # body_length: length of body
    df['body_length'] = df['body'].apply(lambda x: len(str(x)))
    
    return df

# Add features to augmented training data
augmented_train = add_features(augmented_train)

# Add features to test data
test_df = add_features(test_df)

print("Features added successfully!")
print(f"\nAugmented train columns: {augmented_train.columns.tolist()}")
print(f"\nSample of new features:")
print(augmented_train[['body', 'has_url', 'body_length']].head())


Features added successfully!

Augmented train columns: ['body', 'rule', 'subreddit', 'rule_violation', 'has_url', 'body_length']

Sample of new features:
                                                body  has_url  body_length
0  Banks don't want you to know this! Click here ...        0           59
1  SD Stream [ ENG Link 1] (http://www.sportsstre...        1           91
2  Lol. Try appealing the ban and say you won't d...        0           57
3  she will come your home open her legs with  an...        1           75
4  code free tyrande --->>> [Imgur](http://i.imgu...        1          313


In [7]:
# Combine text with [SEP] separators
def combine_text(row):
    """
    Combine text as 'body [SEP] Rule: rule [SEP] Subreddit: subreddit [SEP] Has URL: has_url [SEP] Length: body_length'
    """
    combined = f"{row['body']} [SEP] Rule: {row['rule']} [SEP] Subreddit: {row['subreddit']} [SEP] Has URL: {row['has_url']} [SEP] Length: {row['body_length']}"
    return combined

# Apply text combination to augmented training data
augmented_train['combined_text'] = augmented_train.apply(combine_text, axis=1)

# Apply text combination to test data
test_df['combined_text'] = test_df.apply(combine_text, axis=1)

print("Text combination completed!")
print(f"\nSample of combined text:")
print(augmented_train['combined_text'].iloc[0][:500] + "...")


Text combination completed!

Sample of combined text:
Banks don't want you to know this! Click here to know more! [SEP] Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. [SEP] Subreddit: Futurology [SEP] Has URL: 0 [SEP] Length: 59...


In [8]:
# Split augmented train into 80% train and 20% validation, stratified by rule_violation
X = augmented_train['combined_text']
y = augmented_train['rule_violation']

X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"\nTraining set class distribution:")
print(y_train.value_counts())
print(f"\nValidation set class distribution:")
print(y_val.value_counts())


Training set size: 8116
Validation set size: 2029

Training set class distribution:
rule_violation
1    4071
0    4045
Name: count, dtype: int64

Validation set class distribution:
rule_violation
1    1018
0    1011
Name: count, dtype: int64


In [9]:
# TF-IDF Vectorization
print("Creating TF-IDF vectorizer...")
tfidf_vectorizer = TfidfVectorizer(
    max_features=10000,
    ngram_range=(1, 3),
    stop_words='english'
)

# Fit and transform training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(test_df['combined_text'])

print(f"TF-IDF matrix shape - Train: {X_train_tfidf.shape}, Val: {X_val_tfidf.shape}, Test: {X_test_tfidf.shape}")

# Store results for model comparison
model_results = {}


Creating TF-IDF vectorizer...
TF-IDF matrix shape - Train: (8116, 10000), Val: (2029, 10000), Test: (10, 10000)


In [10]:
# Model 1: Logistic Regression on TF-IDF
print("\n" + "="*50)
print("MODEL 1: LOGISTIC REGRESSION")
print("="*50)

lr_model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'
)

# Train the model
lr_model.fit(X_train_tfidf, y_train)

# Predict on validation set
y_val_pred_lr = lr_model.predict_proba(X_val_tfidf)[:, 1]

# Calculate AUC
lr_auc = roc_auc_score(y_val, y_val_pred_lr)
model_results['Logistic Regression'] = lr_auc

print(f"Logistic Regression Validation AUC: {lr_auc:.4f}")



MODEL 1: LOGISTIC REGRESSION
Logistic Regression Validation AUC: 0.9858


In [11]:
# Model 2: Random Forest Classifier on TF-IDF
print("\n" + "="*50)
print("MODEL 2: RANDOM FOREST")
print("="*50)

rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=10,
    random_state=42,
    class_weight='balanced',
    n_jobs=-1
)

# Train the model
rf_model.fit(X_train_tfidf, y_train)

# Predict on validation set
y_val_pred_rf = rf_model.predict_proba(X_val_tfidf)[:, 1]

# Calculate AUC
rf_auc = roc_auc_score(y_val, y_val_pred_rf)
model_results['Random Forest'] = rf_auc

print(f"Random Forest Validation AUC: {rf_auc:.4f}")



MODEL 2: RANDOM FOREST
Random Forest Validation AUC: 0.9147


In [12]:
# Model 3: XGBoost Classifier on TF-IDF
print("\n" + "="*50)
print("MODEL 3: XGBOOST")
print("="*50)

xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=6,
    random_state=42,
    eval_metric='auc'
)

# Train the model
xgb_model.fit(X_train_tfidf, y_train)

# Predict on validation set
y_val_pred_xgb = xgb_model.predict_proba(X_val_tfidf)[:, 1]

# Calculate AUC
xgb_auc = roc_auc_score(y_val, y_val_pred_xgb)
model_results['XGBoost'] = xgb_auc

print(f"XGBoost Validation AUC: {xgb_auc:.4f}")



MODEL 3: XGBOOST
XGBoost Validation AUC: 0.9479


In [13]:
# Model 4: SVM on TF-IDF
print("\n" + "="*50)
print("MODEL 4: SVM")
print("="*50)

svm_model = SVC(
    kernel='linear',
    probability=True,
    random_state=42,
    class_weight='balanced'
)

# Train the model
svm_model.fit(X_train_tfidf, y_train)

# Predict on validation set
y_val_pred_svm = svm_model.predict_proba(X_val_tfidf)[:, 1]

# Calculate AUC
svm_auc = roc_auc_score(y_val, y_val_pred_svm)
model_results['SVM'] = svm_auc

print(f"SVM Validation AUC: {svm_auc:.4f}")



MODEL 4: SVM
SVM Validation AUC: 0.9859


In [14]:
# Model 5: Simple MLP using PyTorch on TF-IDF embeddings
print("\n" + "="*50)
print("MODEL 5: MLP (PyTorch)")
print("="*50)

# Define MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim1=512, hidden_dim2=256, dropout=0.3):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, 1)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return self.sigmoid(x)

# Convert sparse matrices to dense for PyTorch
X_train_dense = X_train_tfidf.toarray()
X_val_dense = X_val_tfidf.toarray()

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_dense)
y_train_tensor = torch.FloatTensor(y_train.values).unsqueeze(1)
X_val_tensor = torch.FloatTensor(X_val_dense)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model
input_dim = X_train_dense.shape[1]
mlp_model = MLPClassifier(input_dim)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Training loop
mlp_model.train()
for epoch in tqdm(range(10), desc="Training MLP"):
    epoch_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = mlp_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/10, Loss: {epoch_loss/len(train_loader):.4f}")

# Predict on validation set
mlp_model.eval()
with torch.no_grad():
    y_val_pred_mlp = mlp_model(X_val_tensor).numpy().flatten()

# Calculate AUC
mlp_auc = roc_auc_score(y_val, y_val_pred_mlp)
model_results['MLP'] = mlp_auc

print(f"MLP Validation AUC: {mlp_auc:.4f}")



MODEL 5: MLP (PyTorch)


Training MLP:  20%|██        | 2/10 [00:46<03:04, 23.12s/it]

Epoch 2/10, Loss: 0.0848


Training MLP:  40%|████      | 4/10 [01:33<02:19, 23.33s/it]

Epoch 4/10, Loss: 0.0279


...

Training MLP:  80%|████████  | 8/10 [02:52<00:41, 20.54s/it]

Epoch 8/10, Loss: 0.0128


Training MLP: 100%|██████████| 10/10 [03:41<00:00, 22.18s/it]

Epoch 10/10, Loss: 0.0119


MLP Validation AUC: 0.9879


In [15]:
# Compare all models and select the best one
print("\n" + "="*60)
print("MODEL COMPARISON RESULTS")
print("="*60)

for model_name, auc_score in sorted(model_results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model_name:20s}: {auc_score:.4f}")

# Find the best model
best_model_name = max(model_results, key=model_results.get)
best_auc = model_results[best_model_name]

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"🏆 BEST AUC: {best_auc:.4f}")

# Store the best model for predictions
if best_model_name == 'Logistic Regression':
    best_model = lr_model
elif best_model_name == 'Random Forest':
    best_model = rf_model
elif best_model_name == 'XGBoost':
    best_model = xgb_model
elif best_model_name == 'SVM':
    best_model = svm_model
else:  # MLP
    best_model = mlp_model



MODEL COMPARISON RESULTS
MLP                 : 0.9879
SVM                 : 0.9859
Logistic Regression : 0.9858
XGBoost             : 0.9479
Random Forest       : 0.9147

🏆 BEST MODEL: MLP
🏆 BEST AUC: 0.9879


In [16]:
# Generate predictions on test set with unseen rule handling
print(f"\n" + "="*60)
print(f"GENERATING PREDICTIONS WITH BEST MODEL: {best_model_name}")
print("="*60)

# Get unique rules from training and test data
train_rules = set(augmented_train['rule'].unique())
test_rules = set(test_df['rule'].unique())
unseen_rules = test_rules - train_rules

print(f"Training rules: {len(train_rules)}")
print(f"Test rules: {len(test_rules)}")
print(f"Unseen rules: {len(unseen_rules)}")

def handle_unseen_rules(test_df, augmented_train, tfidf_vectorizer):
    """
    Handle unseen rules by averaging similarity to positive/negative examples using TF-IDF cosine similarity
    """
    predictions = []
    
    for idx, row in test_df.iterrows():
        rule = row['rule']
        body = row['body']
        
        if rule in train_rules:
            # Use the best model for seen rules
            if best_model_name == 'MLP':
                X_test_tensor = torch.FloatTensor(X_test_tfidf[idx:idx+1].toarray())
                best_model.eval()
                with torch.no_grad():
                    pred = best_model(X_test_tensor).numpy().flatten()[0]
            else:
                pred = best_model.predict_proba(X_test_tfidf[idx:idx+1])[0, 1]
        else:
            # Handle unseen rules using similarity to examples
            # Get examples with the same rule
            rule_examples = augmented_train[augmented_train['rule'] == rule]
            
            if len(rule_examples) == 0:
                # If no examples found, use global average
                pred = 0.5
            else:
                # Calculate similarity to positive and negative examples
                positive_examples = rule_examples[rule_examples['rule_violation'] == 1]['body'].tolist()
                negative_examples = rule_examples[rule_examples['rule_violation'] == 0]['body'].tolist()
                
                if len(positive_examples) > 0 and len(negative_examples) > 0:
                    # Vectorize current body and examples
                    all_texts = [body] + positive_examples + negative_examples
                    similarity_matrix = tfidf_vectorizer.transform(all_texts)
                    
                    # Calculate cosine similarities
                    current_vector = similarity_matrix[0]
                    positive_vectors = similarity_matrix[1:1+len(positive_examples)]
                    negative_vectors = similarity_matrix[1+len(positive_examples):]
                    
                    # Calculate average similarities
                    pos_similarities = np.array([(current_vector * pos_vec).toarray()[0, 0] / 
                                             (np.linalg.norm(current_vector.toarray()) * np.linalg.norm(pos_vec.toarray()) + 1e-8)
                                             for pos_vec in positive_vectors])
                    neg_similarities = np.array([(current_vector * neg_vec).toarray()[0, 0] / 
                                             (np.linalg.norm(current_vector.toarray()) * np.linalg.norm(neg_vec.toarray()) + 1e-8)
                                             for neg_vec in negative_vectors])
                    
                    avg_pos_sim = np.mean(pos_similarities)
                    avg_neg_sim = np.mean(neg_similarities)
                    
                    # Simple prediction based on similarity ratio
                    if avg_pos_sim + avg_neg_sim > 0:
                        pred = avg_pos_sim / (avg_pos_sim + avg_neg_sim)
                    else:
                        pred = 0.5
                else:
                    pred = 0.5
        
        predictions.append(pred)
    
    return predictions

# Generate predictions
test_predictions = handle_unseen_rules(test_df, augmented_train, tfidf_vectorizer)

print(f"Generated {len(test_predictions)} predictions")
print(f"Prediction range: [{min(test_predictions):.4f}, {max(test_predictions):.4f}]")
print(f"Mean prediction: {np.mean(test_predictions):.4f}")



GENERATING PREDICTIONS WITH BEST MODEL: MLP
Training rules: 2
Test rules: 2
Unseen rules: 0
Generated 10 predictions
Prediction range: [0.0000, 0.9999]
Mean prediction: 0.5505


In [18]:
# Create submission file
submission_df = pd.DataFrame({
    'row_id': test_df['row_id'],
    'rule_violation': test_predictions
})

# Save submission file
submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"\nSubmission file saved to: {submission_path}")
print(f"\nSubmission file shape: {submission_df.shape}")
print(f"\nFirst few rows of submission:")
print(submission_df.head())
print(f"\nLast few rows of submission:")
print(submission_df.tail())



Submission file saved to: submission.csv

Submission file shape: (10, 2)

First few rows of submission:
   row_id  rule_violation
0    2029        0.000813
1    2030        0.512763
2    2031        0.999716
3    2032        0.991736
4    2033        0.999899

Last few rows of submission:
   row_id  rule_violation
5    2034        0.000034
6    2035        0.999705
7    2036        0.000002
8    2037        0.000085
9    2038        0.999858


In [19]:
# Final summary
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)
print(f"Dataset Information:")
print(f"  - Original training samples: {len(train_df)}")
print(f"  - Augmented training samples: {len(augmented_train)}")
print(f"  - Test samples: {len(test_df)}")
print(f"  - Unique rules in training: {len(train_rules)}")
print(f"  - Unique rules in test: {len(test_rules)}")
print(f"  - Unseen rules: {len(unseen_rules)}")

print(f"\nModel Performance:")
for model_name, auc_score in sorted(model_results.items(), key=lambda x: x[1], reverse=True):
    status = "🏆 BEST" if model_name == best_model_name else "  "
    print(f"  {status} {model_name:20s}: {auc_score:.4f}")

print(f"\n🎯 TARGET ACHIEVED: {'✅ YES' if best_auc >= 0.95 else '❌ NO'} (Target: ≥0.95)")
print(f"📊 BEST MODEL: {best_model_name}")
print(f"📈 BEST AUC: {best_auc:.4f}")
print(f"💾 SUBMISSION FILE: {submission_path}")

print("\n" + "="*70)
print("NOTEBOOK EXECUTION COMPLETED SUCCESSFULLY!")
print("="*70)



FINAL SUMMARY
Dataset Information:
  - Original training samples: 2029
  - Augmented training samples: 10145
  - Test samples: 10
  - Unique rules in training: 2
  - Unique rules in test: 2
  - Unseen rules: 0

Model Performance:
  🏆 BEST MLP                 : 0.9879
     SVM                 : 0.9859
     Logistic Regression : 0.9858
     XGBoost             : 0.9479
     Random Forest       : 0.9147

🎯 TARGET ACHIEVED: ✅ YES (Target: ≥0.95)
📊 BEST MODEL: MLP
📈 BEST AUC: 0.9879
💾 SUBMISSION FILE: submission.csv

NOTEBOOK EXECUTION COMPLETED SUCCESSFULLY!
